In [1]:
#libs
import numpy as np
import os
import pandas as pd
import pprint
import csv

#path constants
train_path = '../../data/train'
test_path = '../../data/test'
train_cleaned_path = '../../cleaned_data/train'
test_cleaned_path = '../../cleaned_data/test'

#type constants
vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']

#two label dataframes
train_label_df = pd.read_csv(train_cleaned_path + '/train_label.csv', delimiter = ',', encoding = 'utf-8')
test_label_df = pd.read_csv(test_cleaned_path + '/test_label.csv', delimiter = ',', encoding = 'utf-8')

#label lookup
#example: 
#    label = getLabel('470e538178b3ae1088e9601c4da5f960.csv', train_label_df)

In [1]:
def getLabel(filename, label_df):
    idx = label_df.loc[label_df['sample_file_name'] == filename]
    return idx.iloc[0]['label']

def TraverseFiles(path, vehicle_type, label_df):

#vehicle_type: one string element under vehicle_types = ['ZVe44', 'ZV573', 'ZV63d', 'ZVfd4', 'ZVa9c', 'ZVa78', 'ZV252']
    path = path + '/' + vehicle_type
    #these are variables to calculate traversing progress (DO NOT CHANGE)
    counts_per_percent = int(len(os.listdir(path)) / 100)
    percentage_completion = 0
    counter = 0
    
    file_list=np.empty([0,16])
    for file in os.listdir(path):
        sample_df = pd.read_csv(path + '/' + file, delimiter = ',', encoding = 'utf-8')
        sample_array=np.array(sample_df)
        l=np.array(label_df.loc[label_df['sample_file_name'] == file])
        rep=np.repeat(l,len(sample_array),axis=0)
        sample_array=np.append(sample_array,rep,axis=1)
        file_list=np.concatenate((file_list, sample_array),axis=0)


        
        #belows are to show traversing progress (DO NOT CHANGE)
        counter += 1
        if counter == counts_per_percent:
            counter = 0
            percentage_completion += 1
            print('traversing files under', path, ':', percentage_completion, "%", end="\r", flush=True)
    return file_list

In [279]:
ZVfd4_train=TraverseFiles(train_cleaned_path,'ZVfd4',train_label_df)

In [280]:
ZVfd4_test=TraverseFiles(test_cleaned_path,'ZVfd4',test_label_df)

In [282]:
ZVfd4_train_features=pd.DataFrame({'hours': ZVfd4_train[:, 0], 'engine': ZVfd4_train[:, 1],
                         'oil_pump': ZVfd4_train[:, 2],'pump_pressure': ZVfd4_train[:, 3],
                        'fluid_temperature': ZVfd4_train[:, 4],'concrete_flow': ZVfd4_train[:, 5],
                         'set_pressure': ZVfd4_train[:, 6],'displacement': ZVfd4_train[:, 7],'low_pressure': ZVfd4_train[:, 8]
                             ,'high_pressure': ZVfd4_train[:, 9]
                             ,'positive': ZVfd4_train[:, 11],'negative': ZVfd4_train[:, 12]})
ZVfd4_train_labes=pd.DataFrame({'labels':ZVfd4_train[:,15]})

In [285]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(C=np.inf, solver='lbfgs')
logit.fit(ZVfd4_train_features, ZVfd4_train_labes)

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=inf, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [295]:
prob_test=logit.predict(ZVfd4_test_features)
prob_test
Y_hat=pd.DataFrame({'Yhat':prob_test})
Y_hat


,Yhat
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [310]:
acc=np.sum(np.abs((np.array(Y_hat)-np.array(ZVfd4_test_labes))))/len(Y_hat)
acc

0.4263220188527708